**Исследования уровня NPS телекоммуникационной компании**  


Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым». Компания провела опрос и попросила вас подготовить дашборд с его итогами. Чтобы оценить результаты опроса, оценки разделим на три группы:

 - 9-10 баллов — «cторонники» (англ. promoters);
 - 7-8 баллов — «нейтралы» (англ. passives);
 - 0-6 баллов — «критики» (англ. detractors).  
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

**Цель исследования:**

Определение текущего уровня потребительской лояльности или NPS среди клиентов из России

Ссылка на презентацию
https://disk.yandex.ru/d/IhSpE5CUKIGYdg

Ссылка на DashBoard
https://public.tableau.com/app/profile/kirill5662/viz/YandexProject2/sheet0

In [ ]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [ ]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = 'https://code.s3.yandex.net/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [ ]:
query = """
    Select
        us.user_id as user_id,
        us.lt_day as lt_day,
        Case
            When lt_day < '365'
                then 0
            Else 1
        End as is_new,
        us.age as age,
        us.gender_segment as gender_segment,
        us.os_name as os_name,
        us.cpe_type_name as cpe_type_name,
        l.country as country,
        l.city as city,
        substr(ag.title, 4, 5) as age_segment,
        substr(ts.title, 4, 5) as traffic_segment,
        substr(lt.title, 4, 5) as lifetime_segment,
        us.nps_score as nps_score,
        Case
            When nps_score <= 6
                then 'критики'
            When nps_score = 7 or nps_score = 8
                then 'нейтралы'
            When nps_score = 9 or nps_score = 10
                then 'сторонники'
            Else 1
        End as nps_group
    From user as us
    Left join location as l on us.location_id = l.location_id
    Left join age_segment as ag on us.age_gr_id = ag.age_gr_id
    Left join traffic_segment as ts on us.tr_gr_id = ts.tr_gr_id
    Left join lifetime_segment as lt on us.lt_gr_id = lt.lt_gr_id
"""

In [ ]:
df = pd.read_sql(query, engine)
df.head(20)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,1,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,1,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,1,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники
3,A004TB,4190,1,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонники
4,A004XT,1163,1,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники
5,A005O0,5501,1,42.0,1.0,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики
6,A0061R,1236,1,45.0,0.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,сторонники
7,A009KS,313,0,35.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,сторонники
8,A00AES,3238,1,36.0,1.0,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,сторонники
9,A00F70,4479,1,54.0,1.0,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,сторонники


In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False)